In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

words = open('names.txt').read().splitlines()
# build vocab of characters and mappings to / from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(stoi)

In [2]:
import random
# build the dataset
block_size = 3

def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))  
n2 = int(0.9 * len(words))
Xtrain, Ytrain = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

## same code as in MLP_language_model project, but here i implemented "loss.backward()" from scratch

## structuring code so it look more like pytorch

In [3]:
# func to compare our results 
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [4]:
n_embd = 10
n_hidden = 64


g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
# layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
# layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
#batch norm
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4587


In [5]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtrain.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtrain[ix], Ytrain[ix]

In [6]:
# forward pass
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

#linear layer 1
hprebn = embcat @ W1 + b1 #hidden layer pre-activation

# batch norm layer
bnmeani = 1 / n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff ** 2
bnvar = 1/(n-1) * bndiff2.sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# non linearity
h = torch.tanh(hpreact)

# linear layer 2
logits = h @ W2 + b2

# cross entropy loss
logits_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logits_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum ** -1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# pytorch backward pass for comparision
for p in parameters:
    p.grad = None

for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logits_maxes, logits, h, hpreact, bnraw, bndiff, bndiff2, bnvar, bnvar_inv, bnmeani, hprebn, embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.5874, grad_fn=<NegBackward0>)

In [15]:
print(demb.shape)
print(C.shape)


torch.Size([32, 3, 10])
torch.Size([33, 10])


In [18]:
# backpropagation manually
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n

dprobs = (1.0 / probs) * dlogprobs

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

dcounts_sum = -1 * counts_sum**(-2) * dcounts_sum_inv

dcounts1 = counts_sum_inv * dprobs
dcounts2 = torch.ones_like(counts) * dcounts_sum
dcounts = dcounts1 + dcounts2

dnorm_logits = norm_logits.exp() * dcounts

dlogits_maxes = torch.zeros_like(logits_maxes)
dlogits_maxes = -1 * dnorm_logits.sum(1, keepdim=True)

dlogits1 = torch.ones_like(logits) * dnorm_logits
dlogits2 = torch.zeros_like(logits_maxes)
dlogits2 = F.one_hot(logits.max(dim=1).indices, num_classes=logits.shape[1]) * dlogits_maxes
dlogits = dlogits1 + dlogits2

dh = dlogits @ W2.t()

dW2 = h.t() @ dlogits

db2 = dlogits.sum(dim=0)

dhpreact = (1.0 - h**2) * dh

dbnraw = bngain * dhpreact

dbngain = (bnraw * dhpreact).sum(0, keepdim=True)

dbnbias = dhpreact.sum(0, keepdim=True)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)

dbnvar = -0.5 * (bnvar + 1e-5)**-1.5 * dbnvar_inv

dbndiff2 = torch.ones_like(bndiff2) * 1/(n-1) * dbnvar

dbndiff1_1 = bnvar_inv * dbnraw
dbndiff1_2 = 2 * bndiff * dbndiff2
dbndiff = dbndiff1_1 + dbndiff1_2

dbnmeani = (-1) * dbndiff.sum(0, keepdim=True)

dhprebn1 = dbndiff.clone()
dhprebn2 = (1/n) * torch.ones_like(hprebn) * dbnmeani
dhprebn = dhprebn1 + dhprebn2

dembcat = dhprebn @ W1.t()

dW1 = embcat.t() @ dhprebn

db1 = dhprebn.sum(0, keepdim=True)

demb = dembcat.view(emb.shape)

dC = torch.zeros_like(C)
for row_idx in range(Xb.shape[0]):
    for col_idx in range(Xb.shape[1]):
        ix = Xb[row_idx,col_idx]
        dC[ix] += demb[row_idx,col_idx]

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logits_maxes', dlogits_maxes, logits_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bnraw', dbnraw, bnraw)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)


logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logits_maxes    | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
bnraw           | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
bngain          | exact: False | approximate: True  | maxdiff: 2.3283064365386963e-09
bnbias 